In [ ]:
import dash
from dash import html
import dash_leaflet as dl
from dash.dependencies import Input, Output, State
from dash_extensions.javascript import assign
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Polygon
import numpy as np

from velib_spot_predictor.data.load_data import load_station_information
from velib_spot_predictor.data.geo import CatchmentAreaBuilderColumns

In [ ]:
arrondissements = gpd.read_file("../data/external/arrondissements.geojson")
quartier_paris = gpd.read_file("../data/external/quartier_paris.geojson")
station_information = load_station_information(
    "../data/raw/station_information.json"
)
station_catchment_area = (
    CatchmentAreaBuilderColumns(
        longitude="lon",
        latitude="lat",
    )
    .run(station_information)
    .set_crs("EPSG:4326")
)
station_occupation = gpd.GeoDataFrame(
    np.random.rand(station_information.shape[0], 1),
    geometry=station_catchment_area,
    columns=["occupation"],
    crs="EPSG:4326",
)
station_information = gpd.GeoDataFrame(
    station_information,
    geometry=gpd.points_from_xy(
        station_information["lon"], station_information["lat"]
    ),
    crs="EPSG:4326",
)

In [ ]:
colorscale = ["red", "green"]
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"
color_prop = "occupation"
vmin = 0
vmax = 1
colorbar = dl.Colorbar(
    colorscale=colorscale, width=20, height=150, min=0, max=vmax, unit="/km2"
)
point_to_layer = assign(
    """
    function(feature, latlng, context){
        const {min, max, colorscale, circleOptions, colorProp} = context.hideout;
        const csc = chroma.scale(colorscale).domain([min, max]);
        circleOptions.fillColor = csc(feature.properties[colorProp]);
        return L.circleMarker(latlng, circleOptions);
    }
    """
)

In [ ]:
app = dash.Dash()

arrondissements_layer = dl.GeoJSON(
    data=json.loads(arrondissements.to_json()),
    id="arrondissements",
    options=dict(style=dict(fillOpacity=0)),
    hoverStyle={"weight": 5},
    zoomToBoundsOnClick=True,
)

quartier_paris_layer = dl.GeoJSON(
    data=None,
    id="quartier_paris",
    options=dict(style=dict(color="red", fillOpacity=0)),
    hoverStyle={"weight": 5},
)

station_information_layer = dl.GeoJSON(
    data=json.loads(station_information.to_json()),
    id="station_information",
    options=dict(pointToLayer=point_to_layer),
    cluster=True,
    superClusterOptions=dict(radius=150),
    zoomToBoundsOnClick=True,
    hoverStyle={"stroke": True, "width": 5},
)

occupation_layer = dl.GeoJSON(
    data=None,
    id="occupation",
    options=dict(
        style=dict(weight=1, dashArray="20", color="red", fillOpacity=0.1)
    ),
    hoverStyle={"fillOpacity": 0.5},
)

app.layout = html.Div(
    [
        html.H1("Paris"),
        html.Button("Reset", id="reset"),
        dl.Map(
            [
                dl.TileLayer(),
                arrondissements_layer,
                quartier_paris_layer,
                # station_information_layer,
                occupation_layer,
            ],
            center=[48.8566, 2.3522],
            zoom=12,
            style={"width": "100%", "height": "500px"},
            id="map",
        ),
    ]
)

# @app.callback(
#     Output("quartier_paris", "data"),
#     Input("arrondissements", "clickData"),
#     Input("reset", "n_clicks"),
# )
# def update_feature_info(feature, reset_n_clicks):
#     ctx = dash.callback_context
#     if not ctx.triggered:
#         return None

#     prop_id = ctx.triggered[0]["prop_id"]
#     if "reset-button" in prop_id:
#         return None
#     elif "arrondissements.clickData" in prop_id:
#         return json.loads(quartier_paris[quartier_paris["c_ar"] == feature["properties"]["c_ar"]].to_json())


@app.callback(
    Output("occupation", "data"),
    Input("arrondissements", "clickData"),
    Input("reset", "n_clicks"),
)
def update_feature_info(feature, reset_n_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        return None

    prop_id = ctx.triggered[0]["prop_id"]
    if "reset-button" in prop_id:
        return None
    elif "arrondissements.clickData" in prop_id:
        polygon = Polygon(feature["geometry"]["coordinates"][0])
        intersection = station_occupation.intersection(polygon)

        return json.loads(intersection.to_json())


app.run()